Goal: create functions to mine data for aggregate stats

In [8]:
import pandas as pd

In [ ]:
import os
parent = os.path.abspath('..')
pkl='pkl/allyears_clean'
path = os.path.join(parent,pkl)

In [3]:
df= pd.read_pickle(path)

In [5]:
import sqlite3

In [6]:
conn=sqlite3.connect(os.path.join(parent,'players.db'))

In [14]:
players_df = pd.read_sql('select * from players',conn)

In [ ]:
players_df

In [4]:
d={}
for player in am_players:
    d[player] = df['American_player'].str.contains(player)
for player in euro_players:
    d[player] = df['European_player'].str.contains(player)

In [5]:
import pickle
pickle.dump(d,open('pkl/d','wb'))

In [7]:
#Create shorthands for boolean indexing the dataframe
singles = df['Format']=='Singles'
doubles = df['Format']=='Doubles'
teams = df['Format']=='Teams'
triples = df['Format']=='Triples'
europe_won = df['Europe_won'] == True
america_won = df['Europe_won'] == False

In [10]:
def winlosslist(player, country, df=df):
    
    #singles
    if 'a' in country:
        swin=df[america_won&singles&d[player]].shape[0]
        sloss=df[d[player]&~america_won&singles].shape[0]
        smp=swin+sloss
    else:
        sloss=df[d[player]&america_won&singles].shape[0]
        swin=df[d[player]&~america_won&singles].shape[0]    
        smp=swin+sloss
    swl='{}-{}'.format(swin,sloss)
    try:
        spct=swin*100/(swin+sloss)
    except:
        spct=0
    
    #doubles
    if 'a' in country:
        dwin=df[d[player]&america_won&doubles].shape[0]
        dloss=df[d[player]&~america_won&doubles].shape[0]
        dmp=dwin+dloss
    else:
        dloss=df[d[player]&america_won&doubles].shape[0]
        dwin=df[d[player]&~america_won&doubles].shape[0] 
        dmp=dwin+dloss
    dwl='{}-{}'.format(dwin,dloss)
    try:
        dpct=dwin*100/(dwin+dloss)   
    except:
        dpct=0
    
        #teams
    if 'a' in country:
        twin=df[d[player]&america_won&teams].shape[0]
        tloss=df[d[player]&~america_won&teams].shape[0]
        tmp=twin+tloss
    else:
        tloss=df[d[player]&america_won&teams].shape[0]
        twin=df[d[player]&~america_won&teams].shape[0] 
        tmp=twin+tloss
    twl='{}-{}'.format(twin,tloss)
    try:
        tpct=twin*100/(twin+tloss)   
    except:
        tpct=0
        
        #triples
    if 'a' in country:
        trwin=df[d[player]&america_won&triples].shape[0]
        trloss=df[d[player]&~america_won&triples].shape[0]
        trmp=trwin+trloss
    else:
        trloss=df[d[player]&america_won&triples].shape[0]
        trwin=df[d[player]&~america_won&triples].shape[0] 
        trmp=trwin+trloss
    trwl='{}-{}'.format(trwin,trloss)
    try:
        trpct=round(trwin*100/(trwin+trloss),2)        
    except:
        trpct=0

    #combined
    cwin=dwin+swin+twin+trwin
    closs=dloss+sloss+tloss+trloss
    cmp=cwin+closs
    cwl='{}-{}'.format(cwin,closs)
    cpct=cwin*100/(cwin+closs)
    l=[player,country,smp,swin,sloss,swl,round(spct,2),dmp,dwin,dloss,dwl,round(dpct,2), twl,trwl,cmp, cwin,closs,cwl,round(cpct,2)]
    return(l)

In [11]:
#prepare list of winlosslists for each player to create player stats dataframe
L=[]
for player in am_players:
    try:
        lst = winlosslist(player,'America')
        L.append(lst)
    except: #Zerodivision error:
        print('failure- am: ', player)
#         pass
for player in euro_players:
    try:
        lst = winlosslist(player,'Europe')
        L.append(lst)
    except: #Zerodivision error:
        print('failure -eu ', player)
#         pass

In [ ]:
#create player_stats dataframe
import pandas as pd
colz = 'Player,Team,Smp,Swin,Sloss,Swl,Spct,Dmp,Dwin,Dloss,Dwl,Dpct,Twl,Trwl,Cmp,Cwin,Closs,Cwl,Cpct'.split(',')
player_stats = pd.DataFrame(L, columns = colz)

In [12]:
player_stats.head()

,Player,Team,Smp,Swin,Sloss,Swl,Spct,Dmp,Dwin,Dloss,Dwl,Dpct,Twl,Trwl,Cmp,Cwin,Closs,Cwl,Cpct
0,Johnny Archer,America,30,17,13,17-13,56.67,50,27,23,27-23,54.00,5-4,1-1,91,50,41,50-41,54.95
1,Earl Strickland,America,26,14,12,14-12,53.85,36,24,12,24-12,66.67,3-2,2-0,69,43,26,43-26,62.32
2,Rodney Morris,America,18,10,8,10-8,55.56,29,12,17,12-17,41.38,3-5,2-0,57,27,30,27-30,47.37
3,Shane Van Boening,America,25,10,15,10-15,40.00,25,11,14,11-14,44.00,4-6,0-0,60,25,35,25-35,41.67
4,Corey Deuel,America,13,7,6,7-6,53.85,25,13,12,13-12,52.00,5-2,2-0,47,27,20,27-20,57.45


In [13]:
# save df to pickle
import pickle
pd.to_pickle(player_stats,'pkl/player_stats')